## Q1

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# 파라미터 설정 
learning_rate=0.1
training_epochs=15
batch_size=100

In [4]:
#MNIST data 불러오기 & train-test split
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
#dataset loader에 train_test assign (*batch size, shuffle, drop_last*)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
#Layering(조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), relu 사용, batch normalization)
#각 layer의 hidden node 수: (784,100), (100,100),(100,10)
linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu=torch.nn.ReLU()

bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)
dropout = torch.nn.Dropout(p=0.3)

In [7]:
#xavier initialization 
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 5.4443e-02,  2.2403e-01,  4.8933e-02, -1.4577e-01, -1.8920e-01,
          8.7791e-02, -6.2778e-02, -1.8040e-01, -2.0778e-01,  2.9983e-02,
          1.9306e-01, -2.0536e-02,  1.7146e-01,  1.3683e-01,  2.3379e-02,
         -1.3184e-01, -8.2875e-02,  1.1905e-01, -1.6735e-01,  1.0278e-01,
         -1.4644e-01,  2.8059e-02, -2.0533e-01,  6.6647e-02,  3.9617e-02,
          1.7191e-01, -2.1065e-01, -1.5448e-01, -1.3070e-02,  1.0096e-02,
         -1.9099e-01,  2.0193e-01, -3.7552e-02,  9.0258e-02, -1.0214e-01,
          2.9346e-02, -1.0090e-01,  7.0380e-02,  5.0289e-02, -1.2136e-01,
          6.6057e-02,  1.7333e-01,  2.1797e-02,  2.3108e-01, -2.1180e-02,
          1.7189e-01, -1.6324e-01,  1.3139e-01, -7.1212e-02, -1.3852e-01,
         -1.8221e-02,  1.4865e-01, -1.0223e-01, -5.8830e-02,  6.5457e-02,
          3.0334e-02,  1.3609e-01,  1.5321e-01,  2.1843e-01, -3.0243e-02,
          4.0459e-02, -9.8311e-02,  3.7482e-04, -9.9259e-02,  6.0476e-04,
          7.2781

In [8]:
#torch.nn.Sequential을 이용하여 model 정의하기(linear->batch norm layer -> relu -> dropout)
bn_model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3).to(device)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)

In [10]:
#cost 계산을 위한 변수설정
train_total_batch=len(data_loader)

In [11]:
#Training epoch (cost 값 초기설정(0으로)과 model의 train 설정 꼭 할 것)
bn_model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고 (X, Y 불러오기), back propagation 과 optimizer를 사용하여 loss 최적화
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = bn_model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.507275224
Epoch: 0002 cost = 0.367212802
Epoch: 0003 cost = 0.341287643
Epoch: 0004 cost = 0.316315740
Epoch: 0005 cost = 0.307551503
Epoch: 0006 cost = 0.289353013
Epoch: 0007 cost = 0.279490441
Epoch: 0008 cost = 0.260232151
Epoch: 0009 cost = 0.253042072
Epoch: 0010 cost = 0.259918630
Epoch: 0011 cost = 0.248664975
Epoch: 0012 cost = 0.256456763
Epoch: 0013 cost = 0.241914690
Epoch: 0014 cost = 0.241396412
Epoch: 0015 cost = 0.236278251
Learning finished


In [12]:
#test set으로 모델의 정확도를 검증하는 코드(model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view 를 사용하여 차원 변환할 것/ Y_test를 불러올 때 labels 사용
#accuracy의 초기값 설정(0으로) 꼭 할 것
# Test model and check accuracy
with torch.no_grad():
    bn_model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = bn_model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    #test set에서 random으로 data를 뽑아, label과 prediction을 비교하는 코드
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9302999973297119
Label:  8
Prediction:  3


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## Q2

In [13]:
#Layering(조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), relu 사용, batch normalization)
#각 layer의 hidden node 수: (784,200), (200,150),(150,10)
linear1 = torch.nn.Linear(784, 200, bias=True)
linear2 = torch.nn.Linear(200, 150, bias=True)
linear3 = torch.nn.Linear(150, 10, bias=True)
relu=torch.nn.ReLU()

bn1 = torch.nn.BatchNorm1d(200)
bn2 = torch.nn.BatchNorm1d(150)
dropout = torch.nn.Dropout(p=0.3)

In [14]:
#xavier initialization 
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0732, -0.1406,  0.1581,  ...,  0.0917,  0.1545, -0.0351],
        [ 0.0092, -0.0431,  0.1504,  ..., -0.0383,  0.0031, -0.1123],
        [ 0.0855, -0.1348, -0.1492,  ..., -0.0406,  0.1810,  0.0392],
        ...,
        [ 0.0528, -0.1253, -0.1901,  ..., -0.1609,  0.1377, -0.0422],
        [ 0.1061,  0.0090,  0.1650,  ..., -0.0336, -0.1621,  0.1535],
        [-0.0260, -0.0480, -0.0255,  ..., -0.0064,  0.1248,  0.0955]],
       requires_grad=True)

In [15]:
#torch.nn.Sequential을 이용하여 model 정의하기(linear->batch norm layer -> relu -> dropout)
bn_model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3).to(device)

In [16]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)

In [17]:
#cost 계산을 위한 변수설정
train_total_batch=len(data_loader)

In [18]:
#Training epoch (cost 값 초기설정(0으로)과 model의 train 설정 꼭 할 것)
bn_model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고 (X, Y 불러오기), back propagation 과 optimizer를 사용하여 loss 최적화
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = bn_model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.467206866
Epoch: 0002 cost = 0.334420025
Epoch: 0003 cost = 0.295281202
Epoch: 0004 cost = 0.273213476
Epoch: 0005 cost = 0.256688237
Epoch: 0006 cost = 0.244146928
Epoch: 0007 cost = 0.232485130
Epoch: 0008 cost = 0.226971209
Epoch: 0009 cost = 0.219768703
Epoch: 0010 cost = 0.221625179
Epoch: 0011 cost = 0.207883760
Epoch: 0012 cost = 0.213482812
Epoch: 0013 cost = 0.199974164
Epoch: 0014 cost = 0.193404630
Epoch: 0015 cost = 0.194228932
Learning finished


In [19]:
#test set으로 모델의 정확도를 검증하는 코드(model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view 를 사용하여 차원 변환할 것/ Y_test를 불러올 때 labels 사용
#accuracy의 초기값 설정(0으로) 꼭 할 것
# Test model and check accuracy
with torch.no_grad():
    bn_model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = bn_model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    #test set에서 random으로 data를 뽑아, label과 prediction을 비교하는 코드
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8583999872207642
Label:  5
Prediction:  5
